In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
fmm_fft_df_raw = pd.read_csv('cleaned/cluster_out_fmmfft_32_15.txt', 
                             names = ['N', 'p', 'b', 't', 'time', 'err']).drop(columns=['b','t', 'err'])

fftw_mpi_df_raw = pd.read_csv('cleaned/cluster_out_fftw_mpi.txt', 
                              usecols=[0,1,2,3],
                              names = ['N', 'p', 'run_num','time']).drop(columns='run_num')

In [18]:
fftw_mpi_df = fftw_mpi_df_raw.groupby(['N', 'p'], as_index=False).median()
fmm_fft_df = fmm_fft_df_raw.groupby(['N', 'p'], as_index=False).median()

In [19]:
problem_sizes = sorted(list(set(fftw_mpi_df['N'].unique().tolist()) & set(fmm_fft_df['N'].unique().tolist())))
cores = sorted(list(set(fftw_mpi_df['p'].unique().tolist()) & set(fmm_fft_df['p'].unique().tolist())))

speedup_fftw_df_matrix = []


for n in problem_sizes:
    row_insert = [n]
    for p in cores:
        fmm_fft_rows = fmm_fft_df.loc[(fmm_fft_df['N']==n) & (fmm_fft_df['p']==p)]
        fftw_rows = fftw_mpi_df.loc[(fftw_mpi_df['N']==n) & (fftw_mpi_df['p']==p)]
        if fmm_fft_rows.shape[0] == 0 or fftw_rows.shape[0]==0: 
            row_insert.append(0)
            continue 
        fmm_fft_time = fmm_fft_rows.iloc[0]['time']
        fftw_time = fftw_rows.iloc[0]['time']
        speedup = fftw_time/fmm_fft_time
        row_insert.append(speedup)
    speedup_fftw_df_matrix.append(row_insert)

df_speedup = pd.DataFrame(speedup_fftw_df_matrix, columns=['N'] + [str(c) for c in cores])

In [20]:
df_speedup

,N,2,4,8,16,32,64
0,131072,4.055511,6.358735,23.635608,65.629942,185.225779,222.591260
1,262144,2.201130,7.982958,15.765339,35.412520,101.021134,127.061563
2,524288,1.657863,3.295853,8.982507,25.307929,56.518977,135.160361
3,1048576,1.195690,1.590444,7.784224,14.263219,33.597735,76.880394
4,2097152,0.942768,1.638166,4.291502,8.299855,18.439830,41.512367
5,4194304,0.795690,1.035819,2.237180,6.838749,11.901816,23.763658
6,8388608,0.768763,0.678969,0.947740,3.228464,5.979463,12.304322
7,16777216,0.770656,0.533622,0.684184,1.411404,4.541909,9.294072
8,33554432,1.124776,0.547227,0.527082,0.876461,1.970311,4.148287
9,67108864,1.128540,0.752484,0.464397,0.617491,1.109358,3.323626


In [24]:
fmm_fft_df.loc[(fmm_fft_df['N']==131072)]

,N,p,time
10,131072,2,0.009971
11,131072,4,0.008661
12,131072,8,0.005114
13,131072,16,0.002782
14,131072,32,0.002312
15,131072,64,0.002849


In [25]:
fftw_mpi_df.loc[(fftw_mpi_df['N']==131072)]

,N,p,time
0,131072,1,0.003051
1,131072,2,0.040438
2,131072,4,0.055073
3,131072,8,0.120872
4,131072,16,0.182583
5,131072,32,0.428242
6,131072,64,0.634162


[131072,
 262144,
 524288,
 1048576,
 2097152,
 4194304,
 8388608,
 16777216,
 33554432,
 67108864,
 134217728,
 268435456,
 536870912,
 1073741824]